# Conversando com os Dados

In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
paths = [
    "./data/Aprendizado_de_máquina.pdf",
    "./data/Inteligência_artificial_generativa.pdf",
    "./data/Inteligência_artificial.pdf"
]

pages = []

for path in paths:
    loader = PyPDFLoader(path)
    pages.extend(loader.load())

doc_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = doc_split.split_documents(pages)

for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('./data/', '')
    doc.metadata['doc_id'] = i

In [4]:
directory = './data/chat_retrieval'

embed_model = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embed_model,
    persist_directory=directory
)

## Criando estrutura de conversa

In [5]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model="gpt-4o-mini"
)

In [6]:
from langchain.chains.retrieval_qa.base import RetrievalQA

chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr')
)

In [7]:
pergunta = 'O que é IA? Qual a sua relação com IA Generativa?'
chat_chain.invoke({'query': pergunta})

{'query': 'O que é IA? Qual a sua relação com IA Generativa?',
 'result': 'IA, ou inteligência artificial, é um campo da ciência da computação que busca desenvolver sistemas capazes de realizar tarefas que normalmente requerem inteligência humana, como reconhecimento de fala, compreensão de linguagem, resolução de problemas e tomada de decisões.\n\nA IA generativa é um ramo específico da inteligência artificial que se concentra na criação de conteúdo novo, baseando-se em padrões identificados em conjuntos de dados de treinamento. Isso significa que, com base em entradas, a IA generativa é capaz de criar textos, imagens, áudio, vídeos e até mesmo sequências de código de software. Portanto, a IA generativa é uma aplicação da inteligência artificial que se dedica à geração de novas informações e criações.'}

## Modificando o prompt da chain

In [10]:
from langchain.prompts import PromptTemplate

chain_prompt = PromptTemplate.from_template(
    """
    Utilize o contexto fornecido para responder a pergunta ao final.
    Se você não sabe a resposta, apenas diga que não sabe e não tente inventar a resposta.
    Mantenha a resposa concisa, com no máximo três frases.

    Contexto: {context}

    Pergunta: {question}

    Resposta:
    """
)

In [12]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type_kwargs={'prompt': chain_prompt},
    return_source_documents=True
)

In [14]:
pergunta = 'O que é IA? Qual a sua relação com IA Generativa?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

A inteligência artificial (IA) é um ramo da ciência da computação que busca desenvolver sistemas capazes de simular a inteligência humana, realizando tarefas como reconhecimento de padrões, aprendizado e tomada de decisões. A IA generativa é uma subcategoria da IA que se concentra na criação de conteúdo novo, utilizando modelos aprendidos com dados de treinamento para gerar texto, imagens, áudio e outros tipos de mídia. Portanto, a IA generativa é uma aplicação específica dentro do campo mais amplo da inteligência artificial.


In [16]:
for i in resposta['source_documents']:
    print(i.page_content)
    print("\n===========================\n")

Inteligência artificial generativa
Inteligência artificial generativa ou IA generativa é um ramo da inteligência artificial focado na
criação de conteúdo novo, baseado em padrões identificados nos conjuntos de dados de treinamento. É
possível criar conteúdo como texto, imagens, áudio, vídeo, código de software, podendo ser gerados a
partir de solicitações em linguagem natural, ou até de através de imagens para imagens, e de vídeos para


Moléculas: Os sistemas generativos de IA podem ser treinados em sequências de
aminoácidos ou representações moleculares, como SMILES representando DNA ou
proteínas. Esses sistemas, como o AlphaFold, são usados para previsão da estrutura de
proteínas e descoberta de medicamentos.[20] Conjuntos de dados incluem vários conjuntos
de dados biológicos.
Multimodal: um sistema de IA generativo pode ser construído a partir de vários modelos


acontece a partir da década de 2010, com resultados extramemente fortes no final dessa década e no
início da década de 2

## Outros tipos de Chains

### Stuff

In [17]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='stuff'
)

pergunta = 'O que é IA? Qual a sua relação com IA Generativa?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

IA, ou inteligência artificial, refere-se a sistemas computacionais projetados para realizar tarefas que normalmente requerem inteligência humana, como raciocínio, aprendizado, reconhecimento de padrões e tomada de decisões. A IA generativa é um ramo específico da inteligência artificial que se concentra na criação de conteúdo novo, como texto, imagens, áudio e vídeo, com base em padrões identificados em conjuntos de dados de treinamento. Em outras palavras, enquanto a IA em geral pode incluir uma ampla gama de aplicações, a IA generativa se especializa em produzir novos conteúdos a partir de dados existentes.


### Map Reduce

In [18]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='map_reduce'
)

pergunta = 'O que é IA? Qual a sua relação com IA Generativa?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

IA, ou Inteligência Artificial, refere-se a sistemas ou máquinas que imitam a inteligência humana para realizar tarefas e podem se aprimorar com base nas informações que coletam. A IA Generativa é uma subcategoria da IA que se concentra na geração de novos conteúdos, como texto, imagens ou sons, a partir de padrões que aprendeu em dados existentes. Em resumo, enquanto a IA abrange um conjunto mais amplo de técnicas e aplicações para emular a inteligência humana, a IA Generativa foca especificamente na criação de novos conteúdos.


## Refine

In [19]:
chat_chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=vectordb.as_retriever(search_type='mmr'),
    chain_type='refine'
)

pergunta = 'O que é IA? Qual a sua relação com IA Generativa?'
resposta = chat_chain.invoke({'query': pergunta})
print(resposta['result'])

IA, ou inteligência artificial, refere-se a sistemas ou máquinas que têm a capacidade de executar tarefas que normalmente requerem inteligência humana, como raciocínio, aprendizado, planejamento, compreensão de linguagem, percepção visual e tomada de decisão. Recentemente, a IA tem evoluído rapidamente, especialmente com o surgimento de modelos complexos, como o GPT-3, que possui bilhões de parâmetros, permitindo resultados impressionantes.

A IA Generativa é um ramo específico da inteligência artificial que se concentra na criação de conteúdo novo a partir de padrões identificados em dados de treinamento. Isso inclui a geração de texto, imagens, áudio e até mesmo o desenvolvimento de novos medicamentos, como demonstrado em pesquisas recentes que mostram como a IA pode "sonhar" com drogas inéditas para investigar sua eficácia. Além disso, a IA Generativa também está revolucionando indústrias como o entretenimento, onde é aplicada para reinventar experiências de jogos, oferecendo novas 